In [11]:
import pandas as pd
import os
import json
import sys
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
from calculate import Scorer

In [12]:
df = pd.read_csv('../data/scores.csv')
with open('../data/weeks.json', 'r') as f:
    weeks = json.loads(f.read())

def get_week(row):
    week = row['week']
    year = int(row['date'].split(',')[1].strip())
    if week == 'Super Bowl':
        week = f'{year - 1} Super Bowl'
    if week in weeks:
        return weeks[week]
    return week

df = df.set_index('Unnamed: 0', drop=False)
df['week'] = df.apply(get_week, axis=1)
df['year'] = df.apply(lambda row: row['week'].split()[0], axis=1)
#df['week'] = df.apply(lambda row: row['week'].split(None, 1)[1], axis=1)
df[df['week'] == '1990 Conf Champ']

,Unnamed: 0,Quarter,Time,Tm,Detail,team1_score,team2_score,week,date,team1_city,team2_city,team1_mascot,team2_mascot,team1_original_city,team2_original_city,team1_original_mascot,team2_original_mascot,year
Unnamed: 0,,,,,,,,,,,,,,,,,,
54920,54920,1.0,NaN,Giants,Matt Bahr 28 yard field goal,3,0,1990 Conf Champ,"Jan 27, 1991",New York,Buffalo,Giants,Bills,New York,Buffalo,Giants,Bills,1990
54921,54921,1.0,NaN,Bills,Scott Norwood 23 yard field goal,3,3,1990 Conf Champ,"Jan 27, 1991",New York,Buffalo,Giants,Bills,New York,Buffalo,Giants,Bills,1990
54922,54922,2.0,NaN,Bills,Don Smith 1 yard rush (Scott Norwood kick),3,10,1990 Conf Champ,"Jan 27, 1991",New York,Buffalo,Giants,Bills,New York,Buffalo,Giants,Bills,1990
54923,54923,2.0,NaN,Bills,"Safety, Smith tackled Hostetler in end zone",3,12,1990 Conf Champ,"Jan 27, 1991",New York,Buffalo,Giants,Bills,New York,Buffalo,Giants,Bills,1990
54924,54924,2.0,NaN,Giants,Stephen Baker 14 yard pass from Jeff Hostetler...,10,12,1990 Conf Champ,"Jan 27, 1991",New York,Buffalo,Giants,Bills,New York,Buffalo,Giants,Bills,1990
54925,54925,3.0,NaN,Giants,Ottis Anderson 1 yard rush (Matt Bahr kick),17,12,1990 Conf Champ,"Jan 27, 1991",New York,Buffalo,Giants,Bills,New York,Buffalo,Giants,Bills,1990
54926,54926,4.0,NaN,Bills,Thurman Thomas 31 yard rush (Scott Norwood kick),17,19,1990 Conf Champ,"Jan 27, 1991",New York,Buffalo,Giants,Bills,New York,Buffalo,Giants,Bills,1990
54927,54927,4.0,NaN,Giants,Matt Bahr 21 yard field goal,20,19,1990 Conf Champ,"Jan 27, 1991",New York,Buffalo,Giants,Bills,New York,Buffalo,Giants,Bills,1990
64365,64365,1.0,NaN,49ers,Mike Cofer 47 yard field goal,0,3,1990 Conf Champ,"Jan 20, 1991",New York,San Francisco,Giants,49ers,New York,San Francisco,Giants,49ers,1990


In [13]:

for group_name, df_group in df.groupby(['date', 'team1_city', 'team2_city']):
    rows = df_group.iterrows()
    length = len(list(rows))
    scorer1 = Scorer()
    scorer2 = Scorer()
    
    for row_index, row in df_group.iterrows():
        score1 = scorer1.calculate(float(row['team1_score']), float(row['team2_score']), row['Quarter'], row_index == length - 1)
        score2 = scorer2.calculate(float(row['team2_score']), float(row['team1_score']), row['Quarter'], row_index == length - 1)
        df.at[row['Unnamed: 0'], 'score1'] = score1
        df.at[row['Unnamed: 0'], 'score2'] = score2
df


,Unnamed: 0,Quarter,Time,Tm,Detail,team1_score,team2_score,week,date,team1_city,team2_city,team1_mascot,team2_mascot,team1_original_city,team2_original_city,team1_original_mascot,team2_original_mascot,year,score1,score2
Unnamed: 0,,,,,,,,,,,,,,,,,,,,
0,0,1.0,6:05,Dolphins,Olindo Mare 37 yard field goal,0,3,1999 Week 7,"Oct 24, 1999",Philadelphia,Miami,Eagles,Dolphins,Philadelphia,Miami,Eagles,Dolphins,1999,-14.599862,14.599862
1,1,1.0,1:04,Dolphins,J.J. Johnson 18 yard rush (Olindo Mare kick),0,10,1999 Week 7,"Oct 24, 1999",Philadelphia,Miami,Eagles,Dolphins,Philadelphia,Miami,Eagles,Dolphins,1999,-36.926499,36.926499
2,2,2.0,11:10,Eagles,David Akers 53 yard field goal,3,10,1999 Week 7,"Oct 24, 1999",Philadelphia,Miami,Eagles,Dolphins,Philadelphia,Miami,Eagles,Dolphins,1999,-36.152985,36.152985
3,3,2.0,6:26,Dolphins,Olindo Mare 37 yard field goal,3,13,1999 Week 7,"Oct 24, 1999",Philadelphia,Miami,Eagles,Dolphins,Philadelphia,Miami,Eagles,Dolphins,1999,-50.553671,50.553671
4,4,3.0,3:47,Eagles,Brian Dawkins 67 yard interception return (Nor...,10,13,1999 Week 7,"Oct 24, 1999",Philadelphia,Miami,Eagles,Dolphins,Philadelphia,Miami,Eagles,Dolphins,1999,-34.973128,34.973128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113569,113569,3.0,NaN,Cowboys,Daryl Johnston 1 yard rush (Eddie Murray kick),27,6,1993 Week 15,"Dec 12, 1993",Dallas,Minnesota,Cowboys,Vikings,Dallas,Minnesota,Cowboys,Vikings,1993,148.828879,-148.828879
113570,113570,3.0,NaN,Vikings,Scottie Graham 1 yard rush (Fuad Reveiz kick),27,13,1993 Week 15,"Dec 12, 1993",Dallas,Minnesota,Cowboys,Vikings,Dallas,Minnesota,Cowboys,Vikings,1993,133.582293,-133.582293
113571,113571,4.0,NaN,Cowboys,Eddie Murray 46 yard field goal,30,13,1993 Week 15,"Dec 12, 1993",Dallas,Minnesota,Cowboys,Vikings,Dallas,Minnesota,Cowboys,Vikings,1993,108.055555,-108.055555


In [14]:
df.to_csv('../data/scores_with_index.csv')
